# PkgEvalAnalysis

Latest pkgeval: https://s3.amazonaws.com/julialang-reports/nanosoldier/pkgeval/by_hash/fb69baf_vs_40279f9/report.html

Looking at RangeHelpers.

In [7]:
using DataFrames, Feather

In [27]:
@enum FailureType begin
    # General ones
    Unknown
    InferredFailure
    PrintingChange
    TestAmbiguities
    NewAmbiguity
    ApproxError
    DocTest
    NewExceptionType
    DownloadError
    UnexpectedPass
    BadBoy
    SyntaxError
    VisualRegression
    MissingDep
    BuildError
    UnsatReq
    BSONDataTypeField
    Belapsed
    StderrCheck
    
    # From PkgEval
    MissingDependency
    Inactivity
    MissingBinary
    Untestable
    LogLimit
    TimeLimit
    Syntax
    
    # Special
    AssertionObvious
    StackExport
    SlicesExport
    BLASVectorLike
    SortAmbig
    Uncolon
    JlTypeInf
    QuickSortAlg
    IsFinite
    CassetteLineInfo
    LibBLASNotDefined
    LoadChange
    NormalizeAmbig
    CxxWrap
    STDLIB_BS
    NoMethodAxpy
    UMF_Symbol
    BaseJoin
    SizeDataType
    StatefulTaken
end


In [12]:
# Unpack the data unless it has already been unpacked

#if !isdir("data")
#    run(`tar -xvf data.tar.xz`)
#end
primary  = Feather.read("data/primary_log.feather");
against = Feather.read("data/against_log.feather");

In [39]:
# Join the primary and against data

package_results = leftjoin(primary, against,
     on=:package, makeunique=true, indicator=:source);

In [40]:
# Filter out packages that started failing but didn't start failing when the RNG stream changed

fails = filter(test->test.source == "both" &&
                 test.status != test.status_1 &&
                 test.status in (":fail", ":kill"), package_results)

# Initially we do not know why a package failed
fails.why = fill(Unknown, size(fails, 1));

missing_binary_idx     = fails.reason .== ":binary_dependency"
inactivity_idx         = fails.reason .== ":inactivity"
missing_dependency_idx = fails.reason .== ":missing_dependency"
untestable_idx         = fails.reason .== ":untestable"
log_limit_idx          = fails.reason .== ":log_limit"
time_limit_idx         = fails.reason .== ":time_limit"

# Initially we do not know why a package failed
fails.why = fill(Unknown, size(fails, 1));

fails.why[missing_binary_idx]     .= MissingBinary
fails.why[inactivity_idx]         .= Inactivity
fails.why[missing_dependency_idx] .= MissingDependency
fails.why[untestable_idx]         .= Untestable
fails.why[log_limit_idx]          .= LogLimit
fails.why[time_limit_idx]         .= TimeLimit

nothing

In [41]:
# Some utility functions

query(fails, s) = filter(row -> occursin(s, row[:log]), fails)    

function update_reason!(fails, needle, why)
    idxs = findall(row -> occursin(needle, row), fails.log)
    fails.why[idxs] .= why
    return fails
end

total_unknown(fails) = count(x -> x.why == Unknown, eachrow(fails))

total_unknown (generic function with 1 method)

In [42]:
query(fails, r"Expression: (\N.*?) == ").package

35-element Vector{String}:
 "LatinHypercubeSampling"
 "DiskArrayTools"
 "JuliennedArrays"
 "PatternFolds"
 "SphericalHarmonicArrays"
 "PlasmaEquilibriumToolkit"
 "Quantikz"
 "LessUnitful"
 "DataStructures"
 "DiskArrays"
 "Wasmtime"
 "ForneyLab"
 "QuasiArrays"
 ⋮
 "PlutoHooks"
 "OceanGrids"
 "DisjointCliqueCover"
 "ExprTools"
 "ChainedFixes"
 "FluxArchitectures"
 "YAXArrays"
 "NamedDims"
 "MultivariateOrthogonalPolynomials"
 "ClassicalOrthogonalPolynomials"
 "Cassette"
 "NaiveGAflux"

In [43]:
# Here we pattern match certain test errors and categorize them based on that.

# Generic ones
update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "detect_ambiguities(", TestAmbiguities)

update_reason!(fails, "Expression: all_doctests()", DocTest)
update_reason!(fails, "Error: doctest failure in ", DocTest)

update_reason!(fails, "does not match inferred return type", InferredFailure);
update_reason!(fails, "Expression: isapprox", ApproxError)
update_reason!(fails, r"Expression: (\S*) ≈ (\S*)", ApproxError) 
update_reason!(fails, "Expression: ≈(", ApproxError)
    
update_reason!(fails, "is ambiguous. Candidates:", NewAmbiguity)
update_reason!(fails, " ambiguities found", NewAmbiguity)

update_reason!(fails, "Unsatisfiable requirements detected for package", UnsatReq)



update_reason!(fails, "Log Test Failed at", PrintingChange)
update_reason!(fails, "Expression: occursin(r\"", PrintingChange)
update_reason!(fails, "Expression: startswith(", PrintingChange)
# update_reason!(fails, r"Expression: (\N.*?) == ", PrintingChange)
update_reason!(fails, "Evaluated: occursin(", PrintingChange)
update_reason!(fails, "Evaluated: endswith(", PrintingChange)
update_reason!(fails, "- DIFF ------------------------", PrintingChange)
update_reason!(fails, "LoadError: syntax", SyntaxError)

update_reason!(fails, "Image did not match reference image", VisualRegression)

update_reason!(fails, "      Thrown: ", NewExceptionType)
update_reason!(fails, "The requested URL returned error", DownloadError)
update_reason!(fails, "gzip: stdin: not in gzip format", DownloadError)
update_reason!(fails, "Unexpected Pass", UnexpectedPass)

update_reason!(fails, "Error building ", BuildError)
update_reason!(fails, "isempty(stderr_content)", StderrCheck)


# Specific ones for this release
update_reason!(fails, "Assertion `obvious_subtype == 3", AssertionObvious)
update_reason!(fails, "Base export \"stack\"; uses of it in module", StackExport)
update_reason!(fails, "ArgumentError: only support vector like inputs", BLASVectorLike)
update_reason!(fails, "MethodError: sort!(::", SortAmbig)
update_reason!(fails, "no method matching uncolon", Uncolon)
update_reason!(fails, "undefined symbol: jl_typeinf_begin", JlTypeInf)
update_reason!(fails, "`QuickSortAlg` not defined", QuickSortAlg)
update_reason!(fails, "no method matching isfinite", IsFinite)
update_reason!(fails, "MethodError: no method matching Core.LineInfoNode", CassetteLineInfo)
update_reason!(fails, "`libblas` not defined", LibBLASNotDefined)
update_reason!(fails, "`liblapack` not defined", LibBLASNotDefined)


  

update_reason!(fails, "ERROR: LoadError: ArgumentError: Package ", LoadChange)
update_reason!(fails, "MethodError: normalize(", NormalizeAmbig)
update_reason!(fails, "Failed to precompile CxxWrap", CxxWrap)

update_reason!(fails, "If you want to set `julia_version`", STDLIB_BS)
update_reason!(fails, "no method matching axpby", NoMethodAxpy)

update_reason!(fails, "UndefVarError: `umf", UMF_Symbol)
update_reason!(fails, "function Base.join must be ", BaseJoin)



update_reason!(fails, "Base export \"Slices\"; uses of", SlicesExport)


update_reason!(fails, "type DataType has no field size", SizeDataType)
update_reason!(fails, "type Stateful has no field taken", StatefulTaken)


  




nothing

In [44]:
# Packages that have gotten  

issues_opened = [
    "Checkpointing" # https://github.com/JuliaGPU/GPUCompiler.jl/issues/361
    "CodeInfoTools.jl" # https://github.com/JuliaCompilerPlugins/CodeInfoTools.jl
    "CompatHelperLocal" # https://gitlab.com/aplavin/compathelperlocal.jl/-/issues/1
    "DocumentFunction" # https://github.com/madsjulia/DocumentFunction.jl/issues/6
    "DataStructures "# https://github.com/JuliaCollections/DataStructures.jl/pull/832
    "TightBindingApproximation" # https://github.com/JuliaLang/julia/issues/47476
]

likely_tol = [
];

fixed = [
    "DocStringExtensions" # https://github.com/JuliaDocs/DocStringExtensions.jl/pull/137
    "EndpointRanges" # 
    "AstroImages" # https://github.com/JuliaAstro/AstroImages.jl/pull/35
    "Deductive" # https://github.com/ctrekker/Deductive.jl/pull/28
]


ignored_packages = [
]

Any[]

In [45]:
# Total package failures that we haven't categorized

total_unknown(fails)

92

# Examples

Here are some examples of how one might do queries and categorize errors

In [46]:
# Unknown failures

filter(x -> x.why == Unknown && 
       !(x.package in issues_opened) && 
       !(x.package in likely_tol) && 
       !(x.package in ignored_packages) && 
       !(x.package in fixed) &&
       !(x.reason == ":abort"), 
    fails)

,configuration,package,version,status,reason,duration
,String,String,String,String,String,Float64
1,primary,UsingMerge,"v""0.0.5""",:fail,:unknown,11.58
2,primary,Terminators,"v""0.1.1""",:fail,:test_failures,19.5
3,primary,PrettyPrinting,"v""0.4.0""",:fail,:unknown,27.05
4,primary,LatinHypercubeSampling,"v""1.8.0""",:fail,:test_failures,32.44
5,primary,Malt,"v""0.7.0""",:fail,:test_failures,15.06
6,primary,PatternFolds,"v""0.2.2""",:fail,:test_failures,41.44
7,primary,RectiGrids,missing,:fail,:unknown,0.0
8,primary,Anasol,"v""1.0.1""",:fail,:unknown,44.75
9,primary,TriMatrices,"v""0.2.0""",:fail,:test_failures,34.87


In [32]:
# Categorize

z = []
for i in instances(FailureType)
    n = count(row -> row.why == i, eachrow(fails))
    n > 0 && push!(z, (n, i))
end
sort!(z; rev=true)

25-element Vector{Any}:
 (67, Unknown)
 (31, PrintingChange)
 (10, CxxWrap)
 (9, SizeDataType)
 (9, LoadChange)
 (8, UnsatReq)
 (5, UMF_Symbol)
 (5, Uncolon)
 (5, StackExport)
 (5, TimeLimit)
 (4, JlTypeInf)
 (4, ApproxError)
 (3, StatefulTaken)
 (3, SortAmbig)
 (3, SlicesExport)
 (3, DocTest)
 (2, NormalizeAmbig)
 (2, IsFinite)
 (2, NewExceptionType)
 (2, NewAmbiguity)
 (1, BaseJoin)
 (1, NoMethodAxpy)
 (1, STDLIB_BS)
 (1, BLASVectorLike)
 (1, LogLimit)

In [33]:
filter(x -> x.why == PrintingChange,  fails).package

31-element Vector{String}:
 "SnoopPrecompile"
 "LatinHypercubeSampling"
 "SIMD"
 "PatternFolds"
 "SphericalHarmonicArrays"
 "PlasmaEquilibriumToolkit"
 "Quantikz"
 "DataStructures"
 "Wasmtime"
 "ForneyLab"
 "Replay"
 "LiterateTest"
 "Compat"
 ⋮
 "DiscreteDifferentialGeometry"
 "SphericalHarmonicModes"
 "PlutoHooks"
 "OceanGrids"
 "DisjointCliqueCover"
 "ExprTools"
 "ChainedFixes"
 "YAXArrays"
 "NamedDims"
 "MultivariateOrthogonalPolynomials"
 "ClassicalOrthogonalPolynomials"
 "Cassette"